In [ ]:
import sys
!{sys.executable} -m pip install langchain huggingface_hub

!{sys.executable} -m  pip install bitsandbytes>=0.39.0
!{sys.executable} -m pip install --upgrade accelerate
!{sys.executable} -m pip install --upgrade sentence_transformers
!{sys.executable} -m  pip install --upgrade transformers
!{sys.executable} -m pip install --upgrade trl
!{sys.executable} -m pip install --upgrade peft
!{sys.executable} -m pip install --upgrade sqlite3
!{sys.executable} -m pip install --upgrade pytz
!{sys.executable} -m pip install --upgrade pypdf PyPDF2
!{sys.executable} -m pip install --upgrade langchain_experimental
!{sys.executable} -m pip install --upgrade langchain_community
!{sys.executable} -m pip install --upgrade langchain_openai
!{sys.executable} -m pip install --upgrade openai tiktoken chromadb
!pip install unstructured['pdf'] unstructured
!{sys.executable} -m pip install --upgrade textwrap
!pip install --upgrade --quiet  cohere
!pip install langchain-cohere
!pip install grandalf
!pip install wikipedia
!pip install faiss-cpu
!pip install faiss-gpu
!pip install langchainhub
!pip install -qU langchain-mistralai
!pip install "unstructured[html]"
!pip install bs4
!pip install rank_bm25
!pip install gradio
!pip install langchain_chroma



import gradio as gr
from huggingface_hub import InferenceClient

import os
import time
import sqlite3
import re
from pytz import timezone
import pytz
import datetime
import json
import textwrap
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch
from operator import itemgetter
import os
import sys

import peft
import bitsandbytes
import accelerate
import trl
from accelerate import init_empty_weights
import torch
import pandas as pd

from transformers import AutoConfig, BitsAndBytesConfig
from accelerate.utils import BnbQuantizationConfig, load_and_quantize_model
from accelerate import load_checkpoint_and_dispatch
import accelerate, bitsandbytes

from langchain_mistralai import ChatMistralAI
import getpass
from google.colab import userdata


from langchain import hub
from langchain import PromptTemplate, HuggingFaceHub, LLMChain
from langchain_experimental.agents import create_csv_agent
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain_openai import ChatOpenAI
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from langchain_community.llms import HuggingFaceEndpoint
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.document_loaders import UnstructuredPDFLoader, PyMuPDFLoader
from langchain_community.document_loaders import UnstructuredHTMLLoader
from langchain_community.document_loaders import BSHTMLLoader
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import WebBaseLoader
from langchain.vectorstores import Chroma

from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank
from langchain_core.prompts import (
    ChatPromptTemplate,
    FewShotChatMessagePromptTemplate,
)

from langchain.chains import RetrievalQA
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel, RunnableLambda
from langchain.llms import HuggingFaceHub
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.callbacks.tracers import ConsoleCallbackHandler

from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool
from langchain.tools.retriever import create_retriever_tool
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.agents import AgentType, initialize_agent
from langchain_core.messages import HumanMessage, ToolMessage, SystemMessage, AIMessage
from langchain_core.prompts import (
    ChatPromptTemplate,
    FewShotChatMessagePromptTemplate,
)
from langchain import PromptTemplate, FewShotPromptTemplate

from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_core.tools import Tool
from langchain_experimental.utilities import PythonREPL
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from google.colab import userdata


**LLM instantiation**

In [ ]:
Mistral_TOKEN = userdata.get('mistralapi')
os.environ['MISTRAL_API_KEY'] = Mistral_TOKEN
llmMist = ChatMistralAI(model="mistral-large-latest", temperature =0)

**Embeddings model**

In [ ]:
model_name = "Alibaba-NLP/gte-large-en-v1.5"
model_kwargs = {'device': 'cpu', 'trust_remote_code': True}
encode_kwargs = {'normalize_embeddings': False}

hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

**Create embeddings to use in example selector**

- see data folder for the pickle files listed in below cell
- the instruction set in the pkl files for the example selector look like

[
    {"instruction": "commands for an alteon virt with ip 10.106.34.3 and service https",
     "alteon-command": "/c/slb/virt 1/vip 10.106.34.3/service 443 https"},
    {"instruction": "commands for an alteon virt with id test and ip address 8.89.88.4 and service FTP",
     "alteon-command": "/c/slb/virt test/vip 8.89.88.4/service FTP"},
    {"instruction": "how to create alteon virtual server called arnab with vip 1.1.1.1 and service http",
     "alteon-command": "/c/slb/virt arnab/vip 1.1.1.1/service http"},
    {"instruction": "what are the commands for an alteon virt named idli with vip 10.4.5.3 and service https",
     "alteon-command": "/c/slb/virt idli/vip 10.4.5.3/service 443 https"}
....]

In [ ]:
import pickle
ls = []

files  = ['instructionsetalteon_virt124.pkl', 'instructionsetalteon_virtip124.pkl',
          'instructionsetalteon_virtgrp124.pkl', 'instructionsetalteon_real.pkl',
          'instructionsetalteon_grp124.pkl']

for f in files:

  with open(f, 'rb') as f:
    mastervirt = pickle.load(f)
  df = pd.DataFrame(mastervirt)
  df= df.sample(8)
  tmpls = [ {'instruction': i[0].strip("\n"),'alteon-command':i[1]} for i in df.values]
  ls.extend(tmpls)


**Vectorings all examples and store in Chroma**

In [ ]:
from langchain_core.example_selectors import (
    MaxMarginalRelevanceExampleSelector,
    SemanticSimilarityExampleSelector,
)
to_vectorize = [" ".join(l.values()) for l in ls]
vectorstore = Chroma.from_texts(to_vectorize, hf, metadatas=ls)



**Instantiate retreiver modes for semantic search**

In [ ]:
example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore,
    k=8,
)
example_selector_mmr = MaxMarginalRelevanceExampleSelector(
    vectorstore=vectorstore,
    k=8,
)

**Create Lanchain components and finalize pipeline**

In [ ]:
example_prompt = PromptTemplate(
    input_variables=["instruction", "alteon-command"],
    template="instruction: {instruction}\nalteon-command: {alteon-command}",
)


prefix = """The following are examples of instructions that have been translated to alteon commands.
The response should only include the alteon command for the instruction and nothing else.
"""


similar_prompt = FewShotPromptTemplate(
    example_selector=example_selector_mmr,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix="Instruction: {instruction}\nalteon-command:",
    input_variables=["instruction"],
    example_separator="\n",
)

query = "commands to construct an alteon virt called arnab with vip 8.8.8.9 and service basic-slb"

chain = similar_prompt | llmMist

chain.invoke({"instruction": query}).content


**Gradio web interface**

In [ ]:
import gradio as gr
from huggingface_hub import InferenceClient

def chained_qa(message, history):
  return chain.invoke({"instruction": message}).content

if __name__ == "__main__":
    gr.ChatInterface(
        chained_qa,
        chatbot=gr.Chatbot(height=300),
        textbox=gr.Textbox(placeholder="ALTEON CLI COMMANDER", container=False, scale=7),
        title="Radware Services",
#        description="Ask for Alteon CLI commands",
        theme="soft",
        examples= ["command to create an alteon virtual server called orco, with vip address 10.5.4.33 and service http",
                   "commands to construct an alteon virt called arnab with vip 8.8.8.9 and service basic-slb"],
        cache_examples=False,
        retry_btn=None,
        undo_btn="Delete Previous",
        clear_btn="Clear",
    ).launch(share= True, debug=True)
